In [1]:
import lumipy as lm
import time
from lusidjam import RefreshingToken as rt

# Tutorial 1: Client

Lumipy provides a client class for interfacing to the Luminesce web API and getting results of queries back as pandas DataFrames. Any query you would normally send with the CLI or web GUI will work in the same way here when you pass in a query string. The methods on the client match those on the web api with a couple of additions for convenience.  

## Authentication

Credentials can be supplied as a secrets file or as environment variables (for example, when working in a containerised job) or by supplying a valid token directly (e.g. `RefreshingToken()` in Jupyterhub).

The secrets file is expected to have the following form
```
{
    "api" : {
      "tokenUrl": "<auth token url>",
      "username": "<user name>",
      "password": "<user password>",
      "clientId": "<client id>",
      "clientSecret": "<client secret>",
      "lumiApiUrl": "https://<client>.lusid.com/honeycomb/api/"
    }
}
```

## Build the Client Object

If no path or token is supplied to the constructor the client will fall back on trying to find authentication information in the available environment variables. 

In [2]:
client = lm.get_client(token=rt())

## Sending Queries to Luminesce

The client can be used to send SQL query strings to luminesce either synchronously or asynchronously. 

### Define Query in String

In [3]:
qry_str = """
select 
    ^ 
from 
    Lusid.Instrument
limit 
    5
"""

### Run Synchronously

In [4]:
df = client.query_and_fetch(qry_str)
df

,LusidInstrumentId,ClientInternal,Quasar,ThirdpartyTicker,NTSecurityNumber,Figi,Cins,SecurityNumber,SecurityUniqueQualifier,InstrumentId,AssetId,BloombergGlobalIndentifier,BloombergUniqueId,DisplayName,IsActive
0,LUID_006O2QBE,598479-XSBND-EUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FR NT REDEEM 06/10/2015 EUR 150 - Cls X,True
1,LUID_0025M6KJ,1269110-IEISE-EUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FR NT REDEEM 31/12/2037 EUR 50000 - F1,True
2,LUID_008TL619,5063167-AUBND-AUD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZC CD REDEEM 29/05/2017 AUD - Ser 03,True
3,LUID_00913YPS,4787878-TWTSE-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CW CALL (Yuanta Sec Inv Trst) A 29/12/2016 TWD...,True
4,LUID_0065BSHZ,5928542-HKSEHK-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KO PUT (HSI) E 29/08/2019 HKD 30528,True


### Run Asynchronously

Start the query and get store the execution ID you get back.

In [5]:
ex_id = client.start_query(qry_str)
print(f'Query started with execution id = {ex_id}')

Query started with execution id = 8109aa33-bf77-4556-9e65-fda4a20728de


Check the query's status. This returns the JSON formatted status response as a dictionary.

In [6]:
status = client.get_status(ex_id)
while not status['status'] == 'RanToCompletion':
    status = client.get_status(ex_id)
    time.sleep(1)

Get the data from the completed query using the execution ID. 

In [7]:
df = client.get_result(ex_id)
df

Fetching data... 📡
  Page   0 downloaded (5 rows).
Data fetch finished


,LusidInstrumentId,ClientInternal,Quasar,ThirdpartyTicker,NTSecurityNumber,Figi,Cins,SecurityNumber,SecurityUniqueQualifier,InstrumentId,AssetId,BloombergGlobalIndentifier,BloombergUniqueId,DisplayName,IsActive
0,LUID_005XP8GU,4109907-MYKLSE-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CW CALL (AMMB Holdings Bhd) E 08/09/2014 MYR 7.68,True
1,LUID_0060KS20,742260-THSET-THB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ordinary Shares (Alien Market),True
2,CCY_SAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Saudi riyal,True
3,LUID_005F9I9L,746670-RUMICX-RUB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ordinary Shares,True
4,LUID_0068R7HM,353442-DEFSX-USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3% BD REDEEM 03/10/2010 USD 1000 - `G10`,True


## Catalogs

### Table and Field 

In [ ]:
client.table_field_catalog()

## History

In [9]:
ex_id = client.start_history_query()
print(ex_id)

6a65b498-d39e-4a4f-9ea7-f18e9597cb3c


In [10]:
status = client.get_history_status(ex_id)
print(status['status'])

WaitingForActivation


In [11]:
client.get_history_result(ex_id).iloc[:, :4]

,At,FinishedAt,QueryName,ExecutionId
0,2021-06-15T08:49:01.7254831,2021-06-15 08:49:02.8519488Z,Query,8109aa33-bf77-4556-9e65-fda4a20728de
1,2021-06-15T08:49:00.1776646,2021-06-15 08:49:01.454324Z,Query,e87bb2fa-216a-4483-9428-1b7dc811f0b9
2,2021-06-15T08:48:53.0121788,2021-06-15 08:48:53.8608174Z,Query,b5750585-964e-4dbd-aa3c-2caa6c30cc6d
3,2021-06-15T08:48:48.806968,2021-06-15 08:48:49.0002953Z,Query,28844519-51c4-43cd-90e9-358d743886f6
4,2021-06-15T08:40:47.2736662,2021-06-15 08:40:51.696691Z,Query,26ae315d-303c-42eb-bfef-dbd1dd3092df
...,...,...,...,...
75,2021-06-14T09:17:04.9519764,2021-06-14 09:17:08.2207914Z,Query,e552f1fc-7b4d-4fa1-88fa-e157b55817c6
76,2021-06-14T09:15:40.2298591,2021-06-14 09:15:43.161846Z,Query,abb4e4b9-fc7d-4b5d-8ef1-270ca50f8c43
77,2021-06-14T09:14:43.2251056,2021-06-14 09:14:46.6052457Z,Query,447f8edd-1be0-40aa-a811-15e0e50cd607
78,2021-06-14T09:05:18.9059803,2021-06-14 09:05:22.0677679Z,Query,71d98256-00e1-4454-9247-65e35b5cdb9f
